In [2]:
!pip install OSMPythonTools
!pip install leafmap

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 79.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 9.7 MB/s eta 0:00:00
  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.3.5-py3-none-any.whl size=32716 sha256=4974eea248333b27b81fca9ee8f9d23225cda8b8ffda1759b55f6902d66bdb15
  Stored in directory: /home/malasawedah/.cache/pip/wheels/dc/2e/0b/51582d4741d2847aa6b6cac475561cc83ee65337380d88fb8c
Successfully built OSMPythonTools
Note: you may need to restart the kernel to use updated packages.


In [1]:
from OSMPythonTools.api import Api
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.nominatim import Nominatim
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import Polygon
from shapely.geometry import box

import json

/opt/conda/envs/eurac_env/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_8568/1684037124.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_py

In [22]:
# Extract photovoltaic from OSM
nominatim = Nominatim()
#areaId = nominatim.query('Vienna, Austria').areaId()
areaId = nominatim.query('Austria, Austria').areaId()

overpass = Overpass()
query = overpassQueryBuilder(area=areaId, elementType=['way', 'relation'], selector='"generator:method"="photovoltaic"', includeGeometry=True)
result = overpass.query(query)
photo_voltaic_data = result.toJSON()
len(photo_voltaic_data['elements'])

[overpass] downloading data: [timeout:25][out:json];area(3600016239)->.searchArea;(way["generator:method"="photovoltaic"](area.searchArea);relation["generator:method"="photovoltaic"](area.searchArea);); out body geom;


2158

In [23]:
photo_voltaic_data

{'version': 0.6,
 'generator': 'Overpass API 0.7.61.5 4133829e',
 'osm3s': {'timestamp_osm_base': '2023-11-10T10:26:29Z',
  'timestamp_areas_base': '2023-11-10T05:06:30Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'way',
   'id': 51004997,
   'bounds': {'minlat': 46.8738313,
    'minlon': 10.5090381,
    'maxlat': 46.8743104,
    'maxlon': 10.50971},
   'nodes': [650355084, 650355086, 650355087, 650355085, 650355084],
   'geometry': [{'lat': 46.8742954, 'lon': 10.5090381},
    {'lat': 46.8738313, 'lon': 10.509061},
    {'lat': 46.8738462, 'lon': 10.50971},
    {'lat': 46.8743104, 'lon': 10.5096871},
    {'lat': 46.8742954, 'lon': 10.5090381}],
   'tags': {'addr:city': 'Nauders',
    'addr:country': 'AT',
    'addr:housenumber': '1',
    'addr:postcode': '6543',
    'addr:street': 'Gewerbegebiet',
    'alt_name': 'Personalwohnungen Bergbahnen Nauders',
    'building': 'yes',
    'gen

In [50]:
for item in data:
    if 'building' in item['tags'].keys():
        print("Yes. it is building")
        data.remove(item)


Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building
Yes. it is building


In [52]:
# Convert thjem to GDF then save them as shapefile
gdf = gpd.GeoDataFrame()
for i, element in enumerate(data):
    try:
        #print(element['geometry'])
        coordinates = element['geometry']
        # Extract latitudes and longitudes from the coordinates
        lats = [coord['lat'] for coord in coordinates]
        lons = [coord['lon'] for coord in coordinates]

        # Create a Shapely polygon from the coordinates
        polygon1 = Polygon(zip(lons, lats))
        #print("Polygon WKT:", polygon.wkt)
        gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
        
    except:
        print(i)

#     # Bounding box coordinates
#     minlat = 46.8738313
#     minlon = 10.5090381
#     maxlat = 46.8743104
#     maxlon = 10.50971

#     # Create a Shapely polygon from the bounding box
#     polygon2 = box(minlon, minlat, maxlon, maxlat)

gdf = gpd.GeoDataFrame(gdf, geometry=gdf.geometry)
gdf.crs = 'EPSG:4326'
gdf.to_file('./data/shapefiles/photovoltaic.shp')

/tmp/ipykernel_3643352/3602215071.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_3643352/3602215071.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_3643352/3602215071.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_3643352/3602215071.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_36433

1805
1806
1807
1808
1809
1810
2103
2104


/tmp/ipykernel_3643352/3602215071.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)


In [62]:
with open('./data/photovoltaic_farms.json', 'w') as file:
    json.dump(data, file, indent=2)

In [186]:
# Extract solar source, polygon surrounf the photovoltaic
# overpass = Overpass()
# query = overpassQueryBuilder(area=areaId, elementType=['way', 'relation'], selector='"plant:source"="solar"', includeGeometry=True)
# result = overpass.query(query)
# plant_solar_data = result.toJSON()
# len(plant_solar_data['elements'])

264

In [188]:
# # Convert thjem to GDF then save them as shapefile
# gdf = gpd.GeoDataFrame()
# for i, element in enumerate(plant_solar_data['elements']):
#     try:
#         #print(element['geometry'])
#         coordinates = element['geometry']
#         # Extract latitudes and longitudes from the coordinates
#         lats = [coord['lat'] for coord in coordinates]
#         lons = [coord['lon'] for coord in coordinates]

#         # Create a Shapely polygon from the coordinates
#         polygon1 = Polygon(zip(lons, lats))
#         #print("Polygon WKT:", polygon.wkt)
#         gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
        
#     except:
#         print(i)
#         #
        
# gdf = gpd.GeoDataFrame(gdf, geometry=gdf.geometry)
# gdf.crs = 'EPSG:4326'
# gdf.to_file('./data/plantsolar.shp')

257
258
259
260
261
262
263


/tmp/ipykernel_2575358/1873256692.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_2575358/1873256692.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_2575358/1873256692.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_2575358/1873256692.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append({'id': i, 'geometry': polygon1}, ignore_index=True)
/tmp/ipykernel_25753

In [53]:
# Visulization

#center of each polygon 
gdf['centroid'] = gdf['geometry'].centroid
gdf['Lon'] = gdf['centroid'].x
gdf['Lat'] = gdf['centroid'].y

# Create a map and check the location of each cluster
center_lat, center_lon =  gdf['Lat'].mean(), gdf['Lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add markers for each point on the map
for _, row in gdf.iterrows():
    folium.Marker(location=[row['Lat'], row['Lon']]).add_to(m)

# Display the map
m

/tmp/ipykernel_3643352/1967082073.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid


In [57]:
import leafmap.foliumap as leafmap
leafmap.search_qms(keyword="google")


['qms.basemap.at',
 'qms.basemap.at Orthofoto',
 'qms.EOX::Maps - Sentinel-2 cloudless',
 'qms.GLAD FOREST LOSS 2000-2021',
 'qms.GLAD FOREST LOSS 2018',
 'qms.GLAD FOREST YEAR LOSS 2000-2021',
 'qms.GLAD FOREST YEAR LOSS 2000-2022',
 'qms.GLAD LANDSAT MOSAIC 2018',
 'qms.GLAD LANDSAT MOSAIC 2019',
 'qms.GLAD LANDSAT MOSAIC 2021']

In [55]:
import leafmap
leafmap.__version__

'0.27.0'

In [58]:
m = leafmap.Map(location=[center_lat, center_lon], zoom_start=10)
m.add_xyz_service("qms.EOX::Maps - Sentinel-2 cloudless")

# Add markers for each point on the map
for _, row in gdf.iterrows():
    folium.Marker(location=[row['Lat'], row['Lon']]).add_to(m)

style_function = lambda x: {'fillColor': '#3366cc', 'color': '#3366cc'}
folium.GeoJson(gdf['geometry'], style_function=style_function).add_to(m)
# Display the map
m

### Load the the global PV inventory and filter them